In [1]:
import numpy as np
import random as rn
import findspark
import sys
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")
#findspark.init("D:\\Users\\software\\spark-3.2.1-bin-hadoop3.2\\bin\\..")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 5.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]

In [5]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 1000000
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
#best_global_fitness = max
best_global_fitness=sc.accumulator(max)
mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [6]:
def InitParticles(part,N,M,blf):
    global best_global_fitness
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        posiciones_.append(posicion)
        posiciones= np.array(posiciones_)
        mejor_posiciones_locales_=posiciones_
        mejor_posiciones_locales=np.array(mejor_posiciones_locales_)
        velocidad=[rn.uniform(-100,100) for j in range(N)]
        velocidades_.append(velocidad)
        velocidades= np.array(velocidades_)
        fit = MSE(posiciones[j],objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("best local fitness",blf)
        aux=best_global_fitness.value
        if fit < aux:
            best_global_fitness = fit
            mejor_pos_global = mejor_posiciones_locales[j][:]
    part= np.array([posiciones,velocidades,mejor_posiciones_locales])
    return blf,mejor_pos_global,part 

In [7]:
#def fitnessEval(part,mpl,mpg,blf,bgf, ind):
def fitnessEval(part,mpg,blf, ind):
    filas=part[0:1,ind]
    fit = MSE(filas[0],objetivo)
    if fit< blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        part[2:,ind]=filas
        if fit < bgf:
            best_global_fitness = fit
            mpg = filas[0]
    return part,mpg,blf,bgf

In [8]:
#def posEval(part,mpl,mpg,ind):
def posEval(part,mpg,ind,N):
    velocidades=part[1:2,ind][0]
    mpl=part[2:,ind][0]
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1,ind][0][k]) + c_2*r_2*(mpg[k] - part[0:1,ind][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1,ind][0][k] = part[0:1,ind][0][k] + velocidades[k]
    #return part,velocidades
    return part

In [9]:
#best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)
print("particulas[0:1]-> ",particulas[0:1][0])
print("mejor velocidades-> ",particulas[1:2][0])
print("mejor posiciones locales-> ",particulas[2:][0])
print("mejor_pos_global-> ",mejor_pos_global)
print("best_global_fitness",best_global_fitness)
print("best_local_fitness",best_local_fitness)

particulas[0:1]->  []
mejor velocidades->  []
mejor posiciones locales->  []
mejor_pos_global->  <pyspark.broadcast.Broadcast object at 0x0000021CE067E370>
best_global_fitness 1.7976931348623157e+308
best_local_fitness <pyspark.broadcast.Broadcast object at 0x0000021CE067ED60>


In [10]:
#for i in range (I):
#    for ind in range(m):
#        particulas,mejor_pos_global,best_local_fitness,best_global_fitness=fitnessEval(particulas,mejor_pos_global,best_local_fitness,best_global_fitness,ind)
 #   r_1 = rn.random()
#    r_2 = rn.random()
#    for ind in range(m):
 #       particulas= posEval(particulas,mejor_pos_global,ind,n)
# print("Mejor posición global->",mejor_pos_global)
# print("Posiciones finales->",particulas[0:1][0])

In [11]:
rdd_master= sc.parallelize(particulas)
resultado = rdd_master.map(lambda x:InitParticles(x,n,m,best_local_fitness)).collect()
print("Mejor global fitness-> ",best_global_fitness.value)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (localhost executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\Software\spark-3.3.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 686, in main
  File "C:\Users\Software\spark-3.3.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 678, in process
  File "C:\Users\Software\spark-3.3.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\Software\spark-3.3.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "C:\Users\Usuario\AppData\Local\Temp\ipykernel_16868\2187889368.py", line 2, in <